In [19]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
data_scientist = pd.read_csv("scrape_indeed_data_scientist.csv")

In [5]:
#URL to scrape
url = "http://www.indeed.com/jobs"

In [6]:
#get response from the URL
response = requests.get(url)

In [7]:
soup = BeautifulSoup(response.text, 'html5lib')

In [8]:
#Set the intersted cities to search for
DEFAULT_CITIES = {'New York', 'Chicago', 'San Francisco', 'Austin', 'Seattle',
                  'Los Angeles', 'Philadelphia', 'Atlanta', 'Dallas',
                  'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston',
                  'Miami'}

In [9]:
params = {'q': 'data analyst', 'radius': '100'}
max_results = 100

In [10]:
def _location(result):
    try:
        location = result.find('span', class_='location').get_text().strip()
        return location
    except:
        return None

In [11]:
def _company(result):
    try:
        company = result.find('span', class_='company').get_text().strip()
        return company
    except:
        return None

In [12]:
def _title(result):
    try:
        title = result.find('a', attrs={'data-tn-element': "jobTitle"}).get(
            'title')
        return title
    except:
        return None

In [13]:
def _salary(result):
    try:
        company = result.find('span', class_='salaryText').get_text().strip()
        return company
    except:
        return None

In [14]:
scraped_data = {'location': [],
            'company': [],
            'title': [],
            'salary': []}

for city in DEFAULT_CITIES:
    for start in range(0, max_results, 15):
        url_params = params.copy()
        url_params.update({'l': city, 'start': start})

        response = requests.get(url, params=url_params)
        soup = BeautifulSoup(response.text, 'lxml')
        results = soup.find_all('div', class_='result')
        
        time.sleep(0.5)
        print(response.url)
        
        for result in results:
            scraped_data['location'].append(_location(result))
            scraped_data['company'].append(_company(result))
            scraped_data['title'].append(_title(result))
            scraped_data['salary'].append(_salary(result))
company_info_df = pd.DataFrame(scraped_data)


https://www.indeed.com/jobs?q=data+analyst&radius=100&l=Denver&start=0
https://www.indeed.com/jobs?q=data+analyst&radius=100&l=Denver&start=15
https://www.indeed.com/jobs?q=data+analyst&radius=100&l=Denver&start=30
https://www.indeed.com/jobs?q=data+analyst&radius=100&l=Denver&start=45
https://www.indeed.com/jobs?q=data+analyst&radius=100&l=Denver&start=60
https://www.indeed.com/jobs?q=data+analyst&radius=100&l=Denver&start=75
https://www.indeed.com/jobs?q=data+analyst&radius=100&l=Denver&start=90
https://www.indeed.com/jobs?q=data+analyst&radius=100&l=Houston&start=0
https://www.indeed.com/jobs?q=data+analyst&radius=100&l=Houston&start=15
https://www.indeed.com/jobs?q=data+analyst&radius=100&l=Houston&start=30
https://www.indeed.com/jobs?q=data+analyst&radius=100&l=Houston&start=45
https://www.indeed.com/jobs?q=data+analyst&radius=100&l=Houston&start=60
https://www.indeed.com/jobs?q=data+analyst&radius=100&l=Houston&start=75
https://www.indeed.com/jobs?q=data+analyst&radius=100&l=Hous

In [20]:
data_analyst = company_info_df.drop_duplicates()

In [21]:
data_analyst

,location,company,title,salary
0,None,Fluid Truck Share,Data Engineer,None
1,None,Mindcolor Autism,Lead Board Certified Behavior Analyst (BCBA),None
2,None,SPECTRUM,Network Operations Analyst II - Inventory Mana...,None
3,None,SPECTRUM,Systems Engineer II - Application Analyst,None
4,"Lakewood, CO",MIRACORP,Business Data Analyst (Entry Level),None
...,...,...,...,...
1574,"Wyandanch, NY",Federation of Organizations,Data Analyst,None
1575,"New York, NY 10017 (Turtle Bay area)",Guidepoint Global,"Healthcare Data Analyst, Qsight",None
1576,"New York, NY 10038 (Financial District area)",Endai,Data Analyst Entry Level,None
1577,"New York, NY",Ace-stack LLC,Healthcare Data/Reporting Analyst,$125 a day


In [17]:
data_analyst['location'].unique()

array([None, 'Lakewood, CO', 'Denver, CO', 'Denver, CO 80202 (Lodo area)',
       'Centennial, CO 80112', 'Denver, CO 80206 (Cherry Creek area)',
       'Denver, CO 80203 (North Capitol Hill area)',
       'Denver, CO 80202 (Union Station area)',
       'Colorado Springs, CO 80920 (Briargate area)',
       'Denver, CO 80216 (Five Points area)', 'Englewood, CO 80112',
       'Broomfield, CO', 'Aurora, CO', 'Lafayette, CO',
       'Greenwood Village, CO 80111',
       'Boulder, CO 80302 (Mapleton Hill area)',
       'Denver, CO 80204 (Civic Center area)',
       'Denver, CO 80202 (Central Business District area)',
       'Greenwood Village, CO', 'Boulder, CO 80301 (East Central area)',
       'Colorado Springs, CO 80917 (East Colorado Springs area)',
       'Westminster, CO', 'Longmont, CO 80501 (Pike area)',
       'Denver, CO 80206 (Congress Park area)', 'Louisville, CO 80027',
       'Broomfield, CO 80020', 'Aurora, CO 80011 (Centretech area)',
       'Denver, CO 80211 (Highland area)

In [22]:
data_analyst.to_csv('scrape_indeed_data_analyst.csv', index=False, encoding='utf8')

In [23]:
company_final = pd.merge(data_analyst,data_scientist,how="outer")

In [24]:
company_final.to_csv('scrape_indeed_combined.csv', index=False, encoding='utf8')